<a href="https://colab.research.google.com/github/HOowada/WorkshopArchive/blob/master/EDINET_retry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import requests
import datetime


In [ ]:
import datetime
import requests
import pandas as pd
import zipfile
from tqdm import tqdm_notebook as tqdm
import glob
from bs4 import BeautifulSoup

In [ ]:
!pip install edinet_xbrl

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for edinet_xbrl: filename=edinet_xbrl-0.2.0-py2.py3-none-any.whl size=10849 sha256=e075ac3a2a2c227caa59427ab43de08c0f03f2c23ff01a12f87cbe19a7ab3137
  Stored in directory: /root/.cache/pip/wheels/9f/e5/02/7a2fb1d397b2e5e9badd3ce824287e8469404491af0d346de3
  Created wheel for python-xbrl: filename=python_xbrl-1.1.1-py2.py3-none-any.whl size=10613 sha256=db753485a1f9dd843792065d0ffb3d88f99af91b858e17412ae482032c593231
  Stored in directory: /root/.cache/pip/wheels/6e/24/e6/16c20c0ef5fe7b59c03c893f883aa41ae4fdf30a77d2fd13ad
  Created wheel for ordereddict: filename=ordereddict-1.1-py3-none-any.whl size=3509 sha256=9616e5e97eb5c98d630c8e082bd3a0251355c7cbb5752d5c03b0893aedd97811
  Stored in direct

In [ ]:
from edinet_xbrl.edinet_xbrl_parser import EdinetXbrlParser

# 評価読み込み

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
ratings = pd.read_excel("人的資本比較リスト_最終.xlsx",skiprows=1)

FileNotFoundError: [Errno 2] No such file or directory: '人的資本比較リスト_最終.xlsx'

In [ ]:
ratings.shape

(185, 33)

In [ ]:
ratings["code_w0"] = ratings["Code"].astype(str)+"0"
ratings["code_w0"]

0      45230
1      82520
2      34070
3      27680
4      41830
       ...  
180    79740
181    80150
182    90050
183    90090
184    97350
Name: code_w0, Length: 185, dtype: object

In [ ]:
# ratings

# FY22（提出日2023/03/31~2023/07/01）

## 書類一覧API

In [ ]:
start_date = datetime.date(2023, 3, 25)
end_date = datetime.date(2023, 7, 1)
# end_date = datetime.date(2024, 3, 31)
#決算後3ヶ月以内に有報を提出しなければならない規則から、end_dateを2022/06/30に指定

period = end_date - start_date
period = int(period.days)
day_list = []
for d in range(period):
    day = start_date + datetime.timedelta(days=d)
    day_list.append(day)

day_list.append(end_date)

In [ ]:
str(day_list[0])

'2023-03-25'

In [ ]:
API_KEY = "c32f31ef90834dc29219f7f0bfbc2bc2"

In [ ]:
#結果を格納するための空のリストを用意
report_list =[]
#日付リストの期間に提出された書類のメタデータを取得してjson形式に変換
for day in tqdm(day_list):



    # url = f"https://api.edinet-fsa.go.jp/api/v2/documents.json?date=2023-04-01&type=2&Subscription-Key=c32f31ef90834dc29219f7f0bfbc2bc2"
    url = f"https://api.edinet-fsa.go.jp/api/v2/documents.json?date={str(day)}&type=2&Subscription-Key=c32f31ef90834dc29219f7f0bfbc2bc2"

    # params = {"date": day, "type": 2}

    params = {
        "type": 2,  # 有価証券報告書のタイプ
        # "date": day,  # 任意の日付を設定
    }


    res = requests.get(url, params=params,verify=False,allow_redirects=False)
    # print(res)

    while res.status_code == 403:
        time.sleep(3)
        res = requests.get(url, params=params,verify=False,allow_redirects=False)
        # print(res)

    if res.status_code != 200:
        print(f'response status code other than 200, in {day}')
        continue

    json_data = res.json()
    # print(json_data)

    for num in range(len(json_data["results"])):
        ordinance_code = json_data["results"][num]["ordinanceCode"]
        form_code = json_data["results"][num]["formCode"]
        sec_code = json_data["results"][num]["secCode"]

    #ordinance_code=010かつform_code=030000が有価証券報告書
        if ordinance_code == "010" and  form_code =="030000":
            company_name=json_data["results"][num]["filerName"]
            edi={ '会社名':company_name,
                        '書類名':json_data["results"][num]["docDescription"],
                        'docID':json_data["results"][num]["docID"],
                        '証券コード':json_data["results"][num]["secCode"],
                        '開始時期':json_data["results"][num]["periodStart"],
                        '終了時期':json_data["results"][num]["periodEnd"],
                        '日付': day,
                        'XBRL':json_data["results"][num]["xbrlFlag"]}
            report_list.append(edi)

/var/folders/91/0vfn_9xn47b1dzg556d0kx3h0000gp/T/ipykernel_81423/2268505569.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for day in tqdm(day_list):


  0%|          | 0/99 [00:00<?, ?it/s]

/Users/y.uchino/opt/anaconda3/envs/python388/lib/python3.8/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.edinet-fsa.go.jp'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/y.uchino/opt/anaconda3/envs/python388/lib/python3.8/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.edinet-fsa.go.jp'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/y.uchino/opt/anaconda3/envs/python388/lib/python3.8/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.edinet-fsa.go.jp'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/adv

In [ ]:
doc_list_df = pd.DataFrame(report_list)
doc_list_df.shape

(3323, 8)

In [ ]:
doc_list_df.head()

,会社名,書類名,docID,証券コード,開始時期,終了時期,日付,XBRL
0,ユニ・チャーム株式会社,有価証券報告書－第63期(2022/01/01－2022/12/31),S100QG6V,81130,2022-01-01,2022-12-31,2023-03-27,1
1,株式会社アシックス,有価証券報告書－第69期(2022/01/01－2022/12/31),S100QG4R,79360,2022-01-01,2022-12-31,2023-03-27,1
2,株式会社武蔵カントリー倶楽部,有価証券報告書－第65期(2022/01/01－2022/12/31),S100QFON,None,2022-01-01,2022-12-31,2023-03-27,1
3,富士変速機株式会社,有価証券報告書－第59期(2022/01/01－2022/12/31),S100QG39,62950,2022-01-01,2022-12-31,2023-03-27,1
4,株式会社電算システムホールディングス,有価証券報告書－第2期(2022/01/01－2022/12/31),S100QF5V,40720,2022-01-01,2022-12-31,2023-03-27,1


In [ ]:
doc_list_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3323 entries, 0 to 3322
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   会社名     3323 non-null   object
 1   書類名     3323 non-null   object
 2   docID   3323 non-null   object
 3   証券コード   3049 non-null   object
 4   開始時期    3323 non-null   object
 5   終了時期    3323 non-null   object
 6   日付      3323 non-null   object
 7   XBRL    3323 non-null   object
dtypes: object(8)
memory usage: 207.8+ KB


In [ ]:
doc_list_df.to_csv("company_to_docid.csv")

In [ ]:
# len(doc_list_df["会社名"].unique())

In [ ]:
doc_list_df[doc_list_df["証券コード"]=="27680"]

,会社名,書類名,docID,証券コード,開始時期,終了時期,日付,XBRL
889,双日株式会社,有価証券報告書－第20期(2022/04/01－2023/03/31),S100QYMO,27680,2022-04-01,2023-03-31,2023-06-20,1


In [ ]:
len(doc_list_df["証券コード"].unique())

3047

In [ ]:
ratings["available"] = ratings["code_w0"].isin(doc_list_df["証券コード"].unique())
ratings[ratings["available"]==True].shape

(184, 35)

In [ ]:
# for code in ratings["code_w0"]:


In [ ]:
# ratings[ratings["available"]==True]["docID"]

## pdf書き出し

In [ ]:
## 使用するdocIDをリスト化

In [ ]:
ratings["code_w0"]

0      45230
1      82520
2      34070
3      27680
4      41830
       ...  
180    79740
181    80150
182    90050
183    90090
184    97350
Name: code_w0, Length: 185, dtype: object

In [ ]:
doc_list_df_inlist = doc_list_df[doc_list_df["証券コード"].isin(ratings["code_w0"])]

In [ ]:
docid_list = doc_list_df_inlist["docID"].tolist()

In [ ]:
filename_list = []

for docid in tqdm(docid_list):
    # print("")
    # print(docid)
    # 書類取得APIのエンドポイント
    url = f"https://api.edinet-fsa.go.jp/api/v2/documents/{str(docid)}?type=2&Subscription-Key=c32f31ef90834dc29219f7f0bfbc2bc2"
    # print(url)

    # 書類取得APIのリクエストパラメータ
    params = {
      "type" : 2,
        "Subscription-Key":"c32f31ef90834dc29219f7f0bfbc2bc2"
    }
    # /content/drive/MyDrive/有価証券報告書_pdf出力
    d = doc_list_df[doc_list_df["docID"]==docid]
    company_code = d["証券コード"].values[0]
    company_name = d["会社名"].values[0]
    # 出力ファイル名
    filename = f"pdf_FY22/{company_code}_{company_name}_{docid}.pdf"
    # print(filename)
    # 書類取得APIの呼び出し
    res = requests.get(url, params=params, verify=False)

    # print(res.json())
    # print(headers["Content-Type"])

    if res.status_code == 200:
        # print("good")
        with open(filename, 'wb') as f:
            for chunk in res.iter_content(chunk_size=1024):
                f.write(chunk)
    # else:
    #     print(f"{docid}_error")

/var/folders/91/0vfn_9xn47b1dzg556d0kx3h0000gp/T/ipykernel_81423/1595487127.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for docid in tqdm(docid_list):


  0%|          | 0/184 [00:00<?, ?it/s]

/Users/y.uchino/opt/anaconda3/envs/python388/lib/python3.8/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.edinet-fsa.go.jp'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/y.uchino/opt/anaconda3/envs/python388/lib/python3.8/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.edinet-fsa.go.jp'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/y.uchino/opt/anaconda3/envs/python388/lib/python3.8/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.edinet-fsa.go.jp'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/adv

In [ ]:
# res.url

## xbrl (not done)

In [ ]:
docid_list = doc_list_df_inlist["docID"].tolist()

In [ ]:
filename_list = []

for docid in tqdm(docid_list):
    # 書類取得APIのエンドポイント
    url = f"https://api.edinet-fsa.go.jp/api/v2/documents/{str(docid)}?type=1&Subscription-Key=c32f31ef90834dc29219f7f0bfbc2bc2"
    # print(url)

    # 書類取得APIのリクエストパラメータ
    params = {
      "type" : 1,
        "Subscription-Key":"c32f31ef90834dc29219f7f0bfbc2bc2"
    }

    # 書類取得APIの呼び出し
    res = requests.get(url, params=params, verify=False)
    # print(res.json())
    # print(headers["Content-Type"])
    # ファイルへ出力
    # print(res.status_code)
    if res.status_code == 200:
        filename_list.append(docid + ".zip")
        with open("xbrl_FY22/"+docid+".zip", 'wb') as f:
            for chunk in res.iter_content(chunk_size=1024):
                f.write(chunk)

/var/folders/91/0vfn_9xn47b1dzg556d0kx3h0000gp/T/ipykernel_81423/1142907894.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for docid in tqdm(docid_list):


  0%|          | 0/184 [00:00<?, ?it/s]

/Users/y.uchino/opt/anaconda3/envs/python388/lib/python3.8/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.edinet-fsa.go.jp'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/y.uchino/opt/anaconda3/envs/python388/lib/python3.8/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.edinet-fsa.go.jp'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/y.uchino/opt/anaconda3/envs/python388/lib/python3.8/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.edinet-fsa.go.jp'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/adv

In [ ]:
# filename_list

In [ ]:
for i, docid in enumerate(docid_list):
    print(filename_list[i])
    if zipfile.is_zipfile("xbrl_FY22/"+filename_list[i]) == True:
        with zipfile.ZipFile("xbrl_FY22/"+filename_list[i]) as zip_f:
            zip_f.extractall("xbrl_FY22/"+docid)
    if zipfile.is_zipfile("xbrl_FY22/"+filename_list[i]) == False: #エラー回避
        print("xbrl_FY22/"+filename_list[i] + " is not a Zip File")
        docid_list.remove(docid)

S100QTSR.zip
S100QT8H.zip
S100QWA7.zip
S100QWQB.zip
S100QY1D.zip
S100QXRZ.zip
S100QYKD.zip
S100QYQ4.zip
S100QZ41.zip
S100QYWF.zip
S100QYCY.zip
S100QZ7C.zip
S100QXX3.zip
S100QYMO.zip
S100QYNI.zip
S100QYR0.zip
S100QZT6.zip
S100QZLI.zip
S100QZ74.zip
S100QZRG.zip
S100QZOM.zip
S100QZZ4.zip
S100QZT0.zip
S100QZX0.zip
S100QYHM.zip
S100QZSF.zip
S100R09Z.zip
S100R04O.zip
S100QYDK.zip
S100R01E.zip
S100QXTB.zip
S100QYZA.zip
S100R0QD.zip
S100R0SZ.zip
S100R16T.zip
S100QZOD.zip
S100R1O5.zip
S100R15B.zip
S100R105.zip
S100R0PF.zip
S100R09D.zip
S100R0XG.zip
S100R0PR.zip
S100R1KF.zip
S100R0S2.zip
S100R0HE.zip
S100R1RG.zip
S100R0I6.zip
S100R0D0.zip
S100R24O.zip
S100R1JA.zip
S100R29U.zip
S100R1J4.zip
S100R0WN.zip
S100R1LR.zip
S100R218.zip
S100R1TH.zip
S100R0BK.zip
S100R06O.zip
S100R19K.zip
S100QZPH.zip
S100R30V.zip
S100R2ZO.zip
S100R23Q.zip
S100R1U9.zip
S100R2FO.zip
S100R207.zip
S100R1CK.zip
S100R2ES.zip
S100R12D.zip
S100R248.zip
S100R3AQ.zip
S100R3B5.zip
S100QVYU.zip
S100R38M.zip
S100R29T.zip
S100R3ZX.zip

In [ ]:
# docid_list

In [ ]:
# content

In [ ]:
parser = EdinetXbrlParser()

content = pd.DataFrame(columns=['docID','経営方針','サステナビリティ','事業等のリスク','経営者の分析','重要な契約等','研究開発',
                                # '総資産','経常利益', '売上高'
                               ])


for _docID in tqdm(docid_list):
    #xbrlファイルのパスを指定
    xbrl_path = 'xbrl_FY22/' + _docID + '/XBRL/PublicDoc/*.xbrl'
    # print(_docID)
    xbrl_path = glob.glob(xbrl_path)[0]
    # print(xbrl_path)
    edinet_xbrl_object = parser.parse_file(xbrl_path)



    #タクソノミ要素となるkeyとcontext_refを指定
    #経営方針、経営環境および対処すべき課題等
    key_houshin= 'jpcrp_cor:BusinessPolicyBusinessEnvironmentIssuesToAddressEtcTextBlock'

    # #サステナビリティ
    sust_tagid = xbrl_path.split("/")[4].split("_")[1]
    # print(sust_tagid)
    key_sust =f'jpcrp030000-asr_{sust_tagid}:DisclosureOfSustainabilityRelatedFinancialInformationTextBlock'

    #事業等のリスク
    key_risk ='jpcrp_cor:BusinessRisksTextBlock'

    #経営者による財政状態、経営成績及びキャッシュ・フローの状況の分析
    key_management ='jpcrp_cor:ManagementAnalysisOfFinancialPositionOperatingResultsAndCashFlowsTextBlock'


    key_contracts ='jpcrp_cor:CriticalContractsForOperationTextBlock'

    key_research = 'jpcrp_cor:ResearchAndDevelopmentActivitiesTextBlock'



    #context_refを定義
    context_ref_1 = 'FilingDateInstant'
    context_ref_2 = 'CurrentYearInstant'
    context_ref_3 = 'CurrentYearDuration'

    #例外処理
    # try:
    data_houshin = edinet_xbrl_object.get_data_by_context_ref(key_houshin, context_ref_1).get_value()
    soup_houshin = BeautifulSoup(data_houshin,'html.parser').get_text(strip=True)


    try:
        data_sust = edinet_xbrl_object.get_data_by_context_ref(key_sust, context_ref_1).get_value()
        soup_sust = BeautifulSoup(data_sust,'html.parser').get_text(strip=True)
    except:
        soup_sust = None


    data_risk = edinet_xbrl_object.get_data_by_context_ref(key_risk, context_ref_1).get_value()
    soup_risk = BeautifulSoup(data_risk,'html.parser').get_text(strip=True)

    data_management = edinet_xbrl_object.get_data_by_context_ref(key_management, context_ref_1).get_value()
    soup_management = BeautifulSoup(data_management,'html.parser').get_text(strip=True)


    data_contracts = edinet_xbrl_object.get_data_by_context_ref(key_contracts, context_ref_1).get_value()
    soup_contracts = BeautifulSoup(data_contracts,'html.parser').get_text(strip=True)

    data_research = edinet_xbrl_object.get_data_by_context_ref(key_research, context_ref_1).get_value()
    soup_research = BeautifulSoup(data_research,'html.parser').get_text(strip=True)


    tmp_se = pd.DataFrame([[_docID], [soup_houshin],
                           [soup_sust],
                           [soup_risk],[soup_management],
                           [soup_contracts],[soup_research],
                           # [data_assets],[data_ordinaryincome], [data_netsales]
                          ], index=content.columns).T
    content = pd.concat([content, tmp_se],ignore_index=True)

    # except AttributeError as e:
    #     print(e)


content

/var/folders/91/0vfn_9xn47b1dzg556d0kx3h0000gp/T/ipykernel_81423/2483853652.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for _docID in tqdm(docid_list):


  0%|          | 0/184 [00:00<?, ?it/s]

/Users/y.uchino/opt/anaconda3/envs/python388/lib/python3.8/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


,docID,経営方針,サステナビリティ,事業等のリスク,経営者の分析,重要な契約等,研究開発
0,S100QTSR,１【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在に...,２【サステナビリティに関する考え方及び取組】文中の将来に関する事項は、当連結会計年度末現在に...,３【事業等のリスク】当社は、経済・市場の状況等を含む経営の遂行状況に係るリスクについては、経...,４【経営者による財政状態、経営成績及びキャッシュ・フローの状況の分析】文中の将来に関する事項...,５【経営上の重要な契約等】該当事項はありません。,６【研究開発活動】当社は、「電動機（モータ）とその応用」を事業領域に定め、世界初・世界一にこ...
1,S100QT8H,１ 【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在...,None,２ 【事業等のリスク】有価証券報告書に記載した事業の状況、経理の状況に関する事項のうち、経営...,３ 【経営者による財政状態、経営成績及びキャッシュ・フローの状況の分析】(1) 経営成績等の...,４ 【経営上の重要な契約等】該当事項はありません。,５ 【研究開発活動】該当事項はありません。
2,S100QWA7,１【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在に...,２【サステナビリティに関する考え方及び取組】当社グループのサステナビリティに関する考え方及び...,３【事業等のリスク】有価証券報告書に記載した事業の状況、経理の状況等に関する事項のうち、投資...,４【経営者による財政状態、経営成績及びキャッシュ・フローの状況の分析】当連結会計年度における...,５【経営上の重要な契約等】該当事項はありません。,６【研究開発活動】当社グループは、将来にわたる持続的成長と企業価値の向上を目指し、長期的な視...
3,S100QWQB,１【経営方針、経営環境及び対処すべき課題等】記載事項のうち将来に関する事項は、提出日現在にお...,２【サステナビリティに関する考え方及び取組】（１）考え方・体制当社グループは、企業理念で掲げ...,３【事業等のリスク】［リスク管理への基本方針］当社グループは、システム障害リスク、清算参加者...,４【経営者による財政状態、経営成績及びキャッシュ・フローの状況の分析】１．業績等の概要（１）...,５【経営上の重要な契約等】契約内容相手方の名称契約会社名契約期間備考日経平均株価先物、日経2...,６【研究開発活動】該当事項はありません。
4,S100QY1D,１ 【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在...,２ 【サステナビリティに関する考え方及び取組】文中の将来に関する事項は、当連結会計年度末現在...,３ 【事業等のリスク】Zホールディングス(株)(以下、当社という。)および子会社・関連会社(...,４ 【経営者による財政状態、経営成績及びキャッシュ・フローの状況の分析】１. 財政状態の状況...,５ 【経営上の重要な契約等】１. サービス提供契約契約会社名ヤフー株式会社契約相手先グーグル...,"６ 【研究開発活動】当連結会計年度の研究開発費は39,054百万円であり、主にAIやFint..."
...,...,...,...,...,...,...,...
179,S100QZHY,１ 【経営方針、経営環境及び対処すべき課題等】本項においては、将来に関する事項が含まれていま...,２ 【サステナビリティに関する考え方及び取組】当社のサステナビリティに関する考え方及び取組は...,３ 【事業等のリスク】以下において、トヨタの事業その他のリスクについて、投資家の判断に重要な...,４ 【経営者による財政状態、経営成績及びキャッシュ・フローの状況の分析】（１）経営成績等の状...,５ 【経営上の重要な契約等】2002年１月チェコ共和国において小型乗用車を共同生産するため、...,６ 【研究開発活動】当社は、「クリーンで安全な商品の提供を使命とし、あらゆる企業活動を通じて...
180,S100R9YB,１ 【経営方針、経営環境及び対処すべき課題等】当社グループの経営方針、経営環境及び対処すべき...,２ 【サステナビリティに関する考え方及び取組】文中における将来に関する事項は、当連結会計年度...,３ 【事業等のリスク】当社グループは、グループ全体のリスクマネジメントの基本方針及びリスクマ...,４ 【経営者による財政状態、経営成績及びキャッシュ・フローの状況の分析】（1） 経営成績等の...,５ 【経営上の重要な契約等】該当事項はありません。,６ 【研究開発活動】当社グループは、写真感光材料やドキュメント等の事業で培った材料化学、光学...
181,S100RAQ1,１【経営方針、経営環境及び対処すべき課題等】当社グループの経営方針、経営環境及び対処すべき課...,２【サステナビリティに関する考え方及び取組】当社グループのサステナビリティに関する考え方及び...,３【事業等のリスク】有価証券報告書に記載した事業の状況、経理の状況等に関する事項のうち、経営...,４【経営者による財政状態、経営成績及びキャッシュ・フローの状況の分析】当連結会計年度における...,５【経営上の重要な契約等】特記すべき事項はありません。,６【研究開発活動】当社グループは、素材の開発から出発して製品化を行うことを信条とし、創業以来...
182,S100RAGG,１【経営方針、経営環境及び対処すべき課題等】当社グループの経営方針、経営環境及び対処すべき課...,２【サステナビリティに関する考え方及び取組】１．サステナビリティ全般当社グループは、サステナ...,３【事業等のリスク】当社グループは、経営指針「Our Vision」の下、「中期経営計画（R...,４【経営者による財政状態、経営成績及びキャッシュ・フローの状況の分析】(１）業績等の概要（単...,５【経営上の重要な契約等】当連結会計年度において、経営上の重要な契約等の決定又は締結等はあり...,６【研究開発活動】当社グループの製品とサービスの付加価値化を進め、新たな成長の柱を確立するた...


In [ ]:
content.head(2)

,docID,経営方針,サステナビリティ,事業等のリスク,経営者の分析,重要な契約等,研究開発
0,S100QTSR,１【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在に...,２【サステナビリティに関する考え方及び取組】文中の将来に関する事項は、当連結会計年度末現在に...,３【事業等のリスク】当社は、経済・市場の状況等を含む経営の遂行状況に係るリスクについては、経...,４【経営者による財政状態、経営成績及びキャッシュ・フローの状況の分析】文中の将来に関する事項...,５【経営上の重要な契約等】該当事項はありません。,６【研究開発活動】当社は、「電動機（モータ）とその応用」を事業領域に定め、世界初・世界一にこ...
1,S100QT8H,１ 【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在...,None,２ 【事業等のリスク】有価証券報告書に記載した事業の状況、経理の状況に関する事項のうち、経営...,３ 【経営者による財政状態、経営成績及びキャッシュ・フローの状況の分析】(1) 経営成績等の...,４ 【経営上の重要な契約等】該当事項はありません。,５ 【研究開発活動】該当事項はありません。


In [ ]:
content.shape

(184, 5)

In [ ]:
content.to_csv("csv/FY22_business_status.csv")

In [ ]:
# content["事業等のリスク"]

In [ ]:
# S100QYWF

In [ ]:
# content["経営方針"][0]